In [5]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import wordninja
from wordninja import LanguageModel
import re
import numpy as np
import gzip
import shutil

def preprocess(x):
    x = str(x)
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 去除中文
    x = re.sub(r'[^\w\s]','',x) # 去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '') # 換行符號去除
    return str.strip(x) # 移除左右空白

In [6]:
x = 'xxxapplexxx'
y = 'apple'
x = x[:x.find(y)] + ' ' + y + ' ' + x[x.find(y)+len(y):]
x

'xxx apple xxx'

In [7]:
#df1 = pd.read_excel('../data/台塑企業_ 產品寶典20210303.xlsx',engine='openpyxl').iloc[:,:-1]
#df2 = pd.read_excel('../data/寶典.v3.台塑網.20210901.xlsx',engine='openpyxl')
#df2.columns = df1.columns
#df = df1.append(df2)
df = pd.read_excel('../data/寶典.v4.20211001.xlsx',engine='openpyxl')
df = df.rename(columns={'ITEMNM':'品名','DIVNM':'公司事業部門','CODIV':'公司代號'})
df['品名'] = df['品名'].apply(lambda x:x.strip())

# 載入產品csv
val_df = pd.read_csv('../data/preprocess_for_SQUAD_產品.csv',index_col=0)[['string_X_train','Y_label','EXPNO','from']].dropna(axis=0)

# 合併寶典和SPEC
產品集合 = set(df['品名'].values.tolist() + val_df['Y_label'].values.tolist())
產品集合 = set(val_df['Y_label'].values.tolist())
寶典 = [ preprocess(i) for i in 產品集合]
寶典 = sorted(寶典,key=len)[::-1]
寶典[:5]

['TAIRYLAN ACRYLIC FIBER B',
 'CAUSTIC SODA MICROPEARLS',
 'METHYLTETRAHYDROPHTHALIC',
 'SUPER ABSORBENT POLYMER',
 'MBS IMPACT MODIFIER FOR']

# 資料讀取

In [8]:
df = pd.read_excel('../data/combined_excel.xlsx',index_col=0)
print(df.shape)
df = df.rename(columns={'45A':'string_X_train','SPEC':'Y_label'})
df = df[['string_X_train','Y_label','EXPNO','from']].dropna(subset=['string_X_train'],axis=0)
print(df.shape)
df['from'] = df['from'].apply(lambda x:''.join(x.split('-')[1:]))
df['string_X_train'] = df['string_X_train'].apply(preprocess)
df['Y_label'] = df['Y_label'].apply(lambda x:str(x).split('\n'))
df['p'] = 'not find'
for idx in tqdm(df.index):
    for p in df.loc[idx,'Y_label'] + 寶典:
        if p in df.loc[idx,'string_X_train']:
            df.loc[idx,'p'] = p
            continue
print(df[df['p']!='not find'].shape)
df = df[df['p']!='not find']
df['Y_label'] = df['p']
df = df.drop(['p'],axis=1)

for i in tqdm(df.index):
    x,y = df.loc[i,'string_X_train'],df.loc[i,'Y_label']
    df.loc[i,'string_X_train'] = x[:x.find(y)] + ' ' + y + ' ' + x[x.find(y)+len(y):]
print(df.shape)
df.head(10)

(20737, 17)
(15273, 4)


<ipython-input-8-5d30e2c24e2d>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm(df.index):



(11234, 5)


<ipython-input-8-5d30e2c24e2d>:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df.index):



(11234, 4)


,string_X_train,Y_label,EXPNO,from
0,SHIPMENT OF PVC SUSPENSION RESIN S65D QTY 18 ...,PVC SUSPENSION RESIN,NaN,20210103.xlsx
2,TERMS OF SALE CIF NHAVA SHEVA PORT INDIA70 MT ...,PVC RESIN S60,NaN,20210103.xlsx
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27,20210103.xlsx
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2...,TAISOX 3470,18,20210103.xlsx
6,TERM OF SALE CIF MUNDRA SEAPORT INDIA70 MT OF ...,PVC SUSPENSION RESIN,NaN,20210103.xlsx
7,COMMODITY ...,POLYESTER,25,20210103.xlsx
8,COMMODITY ...,POLYESTER,25,20210103.xlsx
9,COMMODITY ...,POLYESTER,25,20210103.xlsx
10,COMMODITY ...,POLYESTER,25,20210103.xlsx
11,COMMODITY ...,POLYESTER,25,20210103.xlsx


In [9]:
# 起始結束位置標註
def str2index(context,string):
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

ys_lst,ye_lst = [],[]
for i in range(len(df)):
    ys,ye = str2index(df['string_X_train'].values[i],df['Y_label'].values[i])
    ys_lst.append(ys),ye_lst.append(ye)
    
df['string_Y_1'] = ys_lst
df['string_Y_2'] = ye_lst
print(1,df.shape)

df = df[df['Y_label']!=''] # 去掉空值
print(2,df.shape)

df = df.loc[df['string_Y_1']!=-1,:] # 去掉找不到答案的
print(3,df.shape)

df = df.dropna(subset=['string_X_train','Y_label'],axis=0) # 去掉NAN
print(4,df.shape)

df.head(10)

1 (11234, 6)
2 (11234, 6)
3 (11234, 6)
4 (11234, 6)


,string_X_train,Y_label,EXPNO,from,string_Y_1,string_Y_2
0,SHIPMENT OF PVC SUSPENSION RESIN S65D QTY 18 ...,PVC SUSPENSION RESIN,NaN,20210103.xlsx,12,32
2,TERMS OF SALE CIF NHAVA SHEVA PORT INDIA70 MT ...,PVC RESIN S60,NaN,20210103.xlsx,50,63
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27,20210103.xlsx,1,19
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2...,TAISOX 3470,18,20210103.xlsx,17,28
6,TERM OF SALE CIF MUNDRA SEAPORT INDIA70 MT OF ...,PVC SUSPENSION RESIN,NaN,20210103.xlsx,47,67
7,COMMODITY ...,POLYESTER,25,20210103.xlsx,66,75
8,COMMODITY ...,POLYESTER,25,20210103.xlsx,66,75
9,COMMODITY ...,POLYESTER,25,20210103.xlsx,66,75
10,COMMODITY ...,POLYESTER,25,20210103.xlsx,66,75
11,COMMODITY ...,POLYESTER,25,20210103.xlsx,66,75


In [10]:
# 最後檢查 Y1 Y2 位置 和 Y_LABEL要對應上
for i in tqdm(df.index):
    a = df.loc[i,'string_X_train'][df.loc[i,'string_Y_1']:df.loc[i,'string_Y_2']]
    b = df.loc[i,'Y_label']
    assert a==b

<ipython-input-10-4ec65d08a61a>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df.index):


In [11]:
from termcolor import colored

# 實際感受一下
def str2index(context,string):
    if type(string) != str:
        print(string)
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

def color_output(text_input,text_output):
    ys,ye = str2index(text_input,text_output)
    left = text_input[:ys]
    mid = text_output
    right = text_input[ye:]
    print(left,colored(mid,'red'),right)

for j in range(10):
    i = np.random.randint(low=0,high=len(df))
    x = df['string_X_train'].values[i]
    p = df['Y_label'].values[i]
    print(j)
    color_output(x,p)
    print(colored(p,'green'))

0
COMMODITY RELEASE FILM S138B 38UM1320MM3000MQTY 396000SMUNIT PRICE CNY190SMAMOUNT CNY752400COMMODITY RELEASE FILM S138B 38UM1465MM3000MQTY 439500SMUNIT PRICE CNY190SMAMOUNT CNY835050COMMODITY RELEASE FILM S138A 38UM650MM4000MQTY 5200000SMUNIT PRICE CNY200SMAMOUNT CNY10400000COMMODITY RELEASE FILM S138A 38UM540MM4000MQTY 4320000SMUNIT PRICE CNY200SMAMOUNT CNY8640000COMMODITY RELEASE FILM S138A 38UM1080MM4000MQTY 6048000SMUNIT PRICE CNY200SMAMOUNT CNY12096000COMMODITY RELEASE FILM SB38A 38UM540MM4000MQTY 4320000SMUNIT PRICE CNY200SMAMOUNT CNY8640000COMMODITY RELEASE FILM SB38A 38UM650MM4000MQTY 5200000SMUNIT PRICE CNY200SMAMOUNT CNY10400000COMMODITY   PET FILM   BP11 38UM680MM300MQTY 1090KGUNIT PRICE CNY000KGAMOUNT CNY000COMMODITY PET FILM LH216 125UM1380MM200MQTY 4830KGUNIT PRICE CNY000KGAMOUNT CNY000COMMODITY PET FILM CY88 50UM1220MM200MQTY 1710KGUNIT PRICE CNY000KGAMOUNT CNY000TOTAL AMOUNT CNY51763450PRICE TERM CIF HUANGPU CHINA
PET FILM
1
CIF ANY JAPANESE PORT AND OR FOB ANY TAIWAN

In [12]:
# 保存
print(df.shape)
df.to_csv('../data/preprocess_for_SQUAD_產品.csv')
df

(11234, 6)


,string_X_train,Y_label,EXPNO,from,string_Y_1,string_Y_2
0,SHIPMENT OF PVC SUSPENSION RESIN S65D QTY 18 ...,PVC SUSPENSION RESIN,NaN,20210103.xlsx,12,32
2,TERMS OF SALE CIF NHAVA SHEVA PORT INDIA70 MT ...,PVC RESIN S60,NaN,20210103.xlsx,50,63
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27,20210103.xlsx,1,19
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2...,TAISOX 3470,18,20210103.xlsx,17,28
6,TERM OF SALE CIF MUNDRA SEAPORT INDIA70 MT OF ...,PVC SUSPENSION RESIN,NaN,20210103.xlsx,47,67
...,...,...,...,...,...,...
20732,PVC SUSPENSION RESIN S65D FOR 70MT AT USD 12...,PVC SUSPENSION RESIN,NaN,20200712.xlsx,1,21
20733,7000 MT IN 4X20 FT CONTAINERS PVC RESIN S60 AS...,PVC RESIN S60,11,20200712.xlsx,30,43
20734,EPOXY RESIN NPEF178FOB ANY PORT OF TAIWAN,EPOXY RESIN,NaN,20200712.xlsx,1,12
20735,EPOXY RESIN NPEL128CIF SAVANNAH GA,EPOXY RESIN,NaN,20200712.xlsx,1,12
